#Data Pre-Processing and Exporting Libraries

In [ ]:
#necessary Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(color_codes=True)

In [ ]:
# Load the dataset(SWaT)
df = pd.read_csv("/content/drive/MyDrive/VRES/attack-data.csv") #your own file path
df.dropna(inplace=True)

In [ ]:
# extract only these features from df
selected_features = ['FIT201', 'AIT201', 'AIT202', 'AIT203', 'P201', 'P203', 'P205', 'Normal/Attack']
df_selected = df[selected_features]


#Itemset Generation Algorithim


In [ ]:
import numpy as np # linear algebra
import pandas as pd
pd.set_option('display.max_rows', None)
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

#Step 1 : Discreatisation of features for ARM

In [ ]:
# Applying the discretization conditions to the DataFrame with 'df' as the DataFrame name
#df = market_basket_df.copy()

df['FIT201(t)'] = df['FIT201'].apply(lambda x: 'Low' if 0.1 <= x < 2 else 'High' if x >= 2 else 'Medium')
df['AIT201(t)'] = df['AIT201'].apply(lambda x: 'Low' if x < 220 else 'Medium' if 220 <= x < 260 else 'High')
df['AIT202(t)'] = df['AIT202'].apply(lambda x: 'Low' if x <= 6.9 else 'Medium' if 6.9 < x <= 8 else 'High')
df['AIT203(t)'] = df['AIT203'].apply(lambda x: 'Low' if x < 340 else 'Medium' if 340 <= x < 380 else 'High')
df['P201(t)'] = df['P201'].apply(lambda x: 'Low' if x == 1 else 'High' if x == 2 else None)
df['P203(t)'] = df['P203'].apply(lambda x: 'Low' if x == 1 else 'High' if x == 2 else None)
df['P205(t)'] = df['P205'].apply(lambda x: 'Low' if x == 1 else 'High' if x == 2 else None)
df['MV201(t)'] = df['MV201'].apply(lambda x: 'Low' if x == 1 else 'High' if x == 2 else 'Medium')

# Displaying the first few rows of the modified DataFrame
data=df[['FIT201(t)','MV201(t)','AIT201(t)', 'AIT202(t)', 'AIT203(t)', 'P201(t)', 'P203(t)', 'P205(t)']]
data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,FIT201(t),MV201(t),AIT201(t),AIT202(t),AIT203(t),P201(t),P203(t),P205(t)
0,Medium,Low,Low,Low,Low,None,None,None
1,Medium,Low,Low,Low,Low,None,None,None
2,Medium,Low,Low,Low,Low,None,None,None
3,Medium,Low,Low,Low,Low,None,None,None
4,Medium,Low,Low,Low,Low,None,None,None


#Step 2, Applying Association Rule Miing Technique

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules



# one-hot encoding
attack_o = pd.get_dummies(data)

# frequent itemsets using FP-Growth with min_supp=0.00390 which is according to the condition of 1/2^X
frequent_itemsets_o = fpgrowth(attack_o, min_support=0.00390, use_colnames=True)

# association rules
rules_o = association_rules(frequent_itemsets_o, metric='confidence', min_threshold=0.0039)

# print the rules sorted by lift in descending order
#print(rules.sort_values(by='lift', ascending=False))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# extract frequent itemset with length 8 with highest support value

frequent_IGM = frequent_itemsets_o[frequent_itemsets_o['itemsets'].apply(lambda x: len(x)) == 8].sort_values(by='support', ascending=False)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#Generating Transactional Dataset - .= 𝑇(𝑋)∪ 𝑇(𝑋′)

In [ ]:
# prompt: extract
# FIT201	MV201	AIT201	AIT202	AIT203	P201	P203	P205 from df

df_f = df[['FIT201', 'MV201', 'AIT201', 'AIT202', 'AIT203', 'P201', 'P203', 'P205']]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import pandas as pd
import numpy as np

# Assuming original_df is your original DataFrame
# FROZENSET : AIT203(t)_Low', 'AIT202(t)_High', 'P201(t)_Low', 'P203(t)_Low', 'P205(t)_Low', 'AIT201(t)_Low'
# Step 1: Filter the DataFrame based on the specified conditions
filtered_df = df_f[
    (df_f['AIT203'] < 340) &
    (df_f['AIT201'] < 220) &
    (df_f['AIT202'] > 8) &
    (df_f['FIT201'] > 0.1) ].copy()

# Since P201, P203, P205 are to be set to "Low" (value of 1), no need to filter based on these

# Step 2: Perform random sampling for AIT201, AIT202, AIT203 from the filtered dataset

# Set the seed for numpy's random number generator to ensure reproducibility
np.random.seed(42)

# Assuming 'filtered_df' is your DataFrame and 'N' is the number of samples you want
N = 500  # Example: Want 200 samples

# Sample indices from the DataFrame's index
sample_indices = np.random.choice(filtered_df.index, size=N, replace=True)

# Retrieve the sampled rows based on the indices
synthetic_samples = filtered_df.loc[sample_indices]


# Setting P201, P203, P205 to "Low" (1) in the synthetic samples
synthetic_samples['P201'] = 1
synthetic_samples['P203'] = 1
synthetic_samples['P205'] = 1
synthetic_samples['MV201'] = 1
# Optional: If you want to only keep the sampled AIT values and the set P values in the synthetic dataset
synthetic_samples_final = synthetic_samples[['AIT201', 'AIT202', 'AIT203', 'P201', 'P203', 'P205', 'FIT201','MV201']]

synthetic_samples_final.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AIT201,AIT202,AIT203,P201,P203,P205,FIT201,MV201
54134,169.1233,8.625225,305.4601,1,1,1,2.455643,1
54467,168.9951,8.634518,304.7424,1,1,1,2.464102,1
11170,209.4335,8.433287,330.6844,1,1,1,2.797680,1
52254,170.0205,8.623302,305.4345,1,1,1,2.461410,1
51809,170.0526,8.614330,307.4340,1,1,1,2.473457,1


In [ ]:
# prompt: synthetic_samples[AIT201] and df_org = filtered_df[filtered_df['AIT201'] < 220 distriution plot on one another

import matplotlib.pyplot as plt
import seaborn as sns

# Create a figure and set its size
plt.figure(figsize=(10, 6))

# Plot the distribution of AIT201 in synthetic_samples
sns.distplot(synthetic_samples['AIT201'], hist=False, color='r', label='Synthetic Samples')

# Plot the distribution of AIT201 in df_org
sns.distplot(df_org['AIT201'], hist=False, color='b', label='Original Samples')

# Add title and labels
plt.title('Distribution of AIT203 in Synthetic and Original Samples')
plt.xlabel('AIT203')
plt.ylabel('Density')

# Show the plot
plt.legend()
plt.show()


#Evaluation on generated Synthetic Rules

#Discreatise synthetic samples

In [ ]:
# Applying the discretization conditions to the DataFrame with 'df' as the DataFrame name


synthetic_samples_final['FIT201(t)'] = synthetic_samples_final['FIT201'].apply(lambda x: 'Low' if 0.1 <= x < 2 else 'High' if x >= 2 else 'Medium')
synthetic_samples_final['AIT201(t)'] = synthetic_samples_final['AIT201'].apply(lambda x: 'Low' if x < 220 else 'Medium' if 220 <= x < 260 else 'High')
synthetic_samples_final['AIT202(t)'] = synthetic_samples_final['AIT202'].apply(lambda x: 'Low' if x <= 6.9 else 'Medium' if 6.9 < x <= 8 else 'High')
synthetic_samples_final['AIT203(t)'] = synthetic_samples_final['AIT203'].apply(lambda x: 'Low' if x < 340 else 'Medium' if 340 <= x < 380 else 'High')
synthetic_samples_final['P201(t)'] = synthetic_samples_final['P201'].apply(lambda x: 'Low' if x == 1 else 'High' if x == 2 else None)
synthetic_samples_final['P203(t)'] = synthetic_samples_final['P203'].apply(lambda x: 'Low' if x == 1 else 'High' if x == 2 else None)
synthetic_samples_final['P205(t)'] = synthetic_samples_final['P205'].apply(lambda x: 'Low' if x == 1 else 'High' if x == 2 else None)
synthetic_samples_final['MV201(t)'] = synthetic_samples_final['MV201'].apply(lambda x: 'Low' if x == 1 else 'High' if x == 2 else 'Medium')

# Displaying the first few rows of the modified DataFrame
data_synthetic = synthetic_samples_final[['FIT201(t)','MV201(t)','AIT201(t)', 'AIT202(t)', 'AIT203(t)', 'P201(t)', 'P203(t)', 'P205(t)']]
data_synthetic.head()

#Association rules on synthetic dataset

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# one-hot encoding
attack_synthetic = pd.get_dummies(data_synthetic)

# frequent itemsets using FP-Growth with min_supp=0.00390 which is according to the condition of 1/2^X
frequent_itemsets_syn = apriori(attack_synthetic , min_support=1, use_colnames=True)

# association rules
rules_syn = association_rules(frequent_itemsets_syn, metric='confidence', min_threshold=1)

# print the rules sorted by lift in descending order
#print(rules.sort_values(by='lift', ascending=False))

#Association rule mining with original dataset

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules



# one-hot encoding
attack_ori = pd.get_dummies(data)

# frequent itemsets using FP-Growth with min_supp=0.00390 which is according to the condition of 1/2^X
frequent_itemsets_ori = fpgrowth(attack_ori, min_support=0.70, use_colnames=True)

# association rules
rules_ori = association_rules(frequent_itemsets_ori ,metric='confidence', min_threshold=0.70)

# print the rules sorted by lift in descending order
#print(rules.sort_values(by='lift', ascending=False))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# common rules in synthetic and real dataset

# Find the intersection of rule antecedents and consequents in both datasets
common_antecedents = set(rules_ori['antecedents']) & set(rules_syn['antecedents'])
common_consequents = set(rules_ori['consequents']) & set(rules_syn['consequents'])

# Count the number of rules with common antecedents or consequents
count_common_rules = len(common_antecedents) + len(common_consequents)

# Print the result
print("Number of common rules:", count_common_rules)


#Predictive modelling technique

#For predictive modelling our synthetic samples are augmented with the the real attack samples and now this data is compared with the real dataset.

In [ ]:
# prompt: synthetic_samples_final select few columns

syn_samples = synthetic_samples_final[['AIT201', 'AIT202', 'AIT203', 'FIT201','MV201','P201','P203','P205']]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# prompt: augment data_synthetic with data

augmented_data = pd.concat([syn_samples, df_new], ignore_index=True)
augmented_data.count() #~55000
augmented_data.head()

In [ ]:
#df_normal = pd.read_csv('/content/drive/MyDrive/VRES/ARM_N_S.csv')
#df_N = df_normal.drop('P101', axis=1, inplace=True)
#df_normal.count()
import pandas as pd
#ARM_unbalanced = pd.read_csv('/content/drive/MyDrive/VRES/extracted_dataset.csv')
#ARM_unbalanced.drop([' Timestamp', 'P101',' P202', ' P204', 'P206'], axis=1, inplace=True)
ARM_unbalanced.head()


,FIT201,MV201,AIT201,AIT202,AIT203,P201,P203,P205,Normal/Attack
0,2.445391,2,262.0161,8.396437,328.6337,1,2,2,Normal
1,2.445391,2,262.0161,8.396437,328.6337,1,2,2,Normal
2,2.442316,2,262.0161,8.394514,328.6337,1,2,2,Normal
3,2.442316,2,262.0161,8.394514,328.6337,1,2,2,Normal
4,2.443085,2,262.0161,8.394514,328.6337,1,2,2,Normal


In [ ]:
unbalanced_samples = ARM_unbalanced.iloc[:100000]
unbalanced_samples.count()

FIT201           100000
 MV201           100000
 AIT201          100000
AIT202           100000
AIT203           100000
 P201            100000
P203             100000
P205             100000
Normal/Attack    100000
dtype: int64

In [ ]:
# prompt: extract 55000 normal samples only

#normal_samples = df_normal[df_normal['Normal/Attack'] == 1]
normal_samples = df_normal.iloc[:55000]
normal_samples.count()

#concatenated dataset with real and synthetic samples

In [ ]:
# prompt: augment attack and normal samples

balanced_dataset = pd.concat([normal_samples, augmented_data], ignore_index=True)
balanced_dataset.count()


#Predictive modelling

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load your data into a DataFrame
# df = pd.read_csv('path_to_your_data.csv')

# Prepare feature matrix X and target vector y
X = unbalanced_samples.drop('Normal/Attack', axis=1)
y = unbalanced_samples['Normal/Attack']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize models
log_reg = LogisticRegression(random_state=42)
svm_model = SVC(random_state=42)
random_forest = RandomForestClassifier(random_state=42)

# Train models
log_reg.fit(X_train_scaled, y_train)
svm_model.fit(X_train_scaled, y_train)
random_forest.fit(X_train_scaled, y_train)

# Predict on test data
log_reg_preds = log_reg.predict(X_test_scaled)
svm_preds = svm_model.predict(X_test_scaled)
random_forest_preds = random_forest.predict(X_test_scaled)

# Evaluate accuracy
log_reg_acc = accuracy_score(y_test, log_reg_preds)
svm_acc = accuracy_score(y_test, svm_preds)
random_forest_acc = accuracy_score(y_test, random_forest_preds)

print("Accuracy of Logistic Regression: ", log_reg_acc)
print("Accuracy of SVM: ", svm_acc)
print("Accuracy of Random Forest: ", random_forest_acc)


Accuracy of Logistic Regression:  0.93848
Accuracy of SVM:  0.9434
Accuracy of Random Forest:  0.99824


#Predictive modelling-real

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load your data into a DataFrame
# df = pd.read_csv('path_to_your_data.csv')  # Uncomment this line if you're loading from a CSV file

# For demonstration, let's assume your DataFrame is named 'df' and is already loaded

# Prepare feature matrix X and target vector y
X = ARM_unbalanced.drop('Normal/Attack', axis=1)
y = ARM_unbalanced['Normal/Attack']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize models
log_reg = LogisticRegression(random_state=42)
svm_model = SVC(random_state=42)
random_forest = RandomForestClassifier(random_state=42)

# Train models
log_reg.fit(X_train_scaled, y_train)
svm_model.fit(X_train_scaled, y_train)
random_forest.fit(X_train_scaled, y_train)

# Predict on test data
log_reg_preds = log_reg.predict(X_test_scaled)
svm_preds = svm_model.predict(X_test_scaled)
random_forest_preds = random_forest.predict(X_test_scaled)

# Evaluate accuracy
log_reg_acc = accuracy_score(y_test, log_reg_preds)
svm_acc = accuracy_score(y_test, svm_preds)
random_forest_acc = accuracy_score(y_test, random_forest_preds)

print("Accuracy of Logistic Regression: ", log_reg_acc)
print("Accuracy of SVM: ", svm_acc)
print("Accuracy of Random Forest: ", random_forest_acc)
